pip install tensorflow --user

In [1]:
import tensorflow as tf; 
print(tf.__version__)
import random
random.seed(123)

2.12.0


In [2]:
# Librerias
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np
import pandas as pd
from ipynb.fs.full.functions_models import Denormalization
from sklearn.metrics import mean_squared_error
from math import sqrt

# Ventana histórico 30 minutos - Pred 15 minutos

## Data Real

In [3]:
Archivo = 'Ventana2/Real/Ventana15.xlsx'
ventana = 2

nombre1="VentanaPredicciones/Real-Modelo1-VH2-P15.xlsx"
nombre2="VentanaPredicciones/Real-Modelo2-VH2-P15.xlsx"
nombre3="VentanaPredicciones/Real-Modelo3-VH2-P15.xlsx"
nombre4="VentanaPredicciones/Real-Modelo4-VH2-P15.xlsx"
nombre5="VentanaPredicciones/Real-Modelo5-VH2-P15.xlsx"
nombre6="VentanaPredicciones/Real-Modelo6-VH2-P15.xlsx"

In [4]:
sheet = 'Training_esc'
training = pd.read_excel(Archivo, sheet, engine='openpyxl')
(training).head()
X_train = training[['X1', 'X2']]
y_train = training[['Y1']]

X_train = X_train.values
y_train = y_train.values

In [5]:
sheet = 'Validation_esc'
Validation = pd.read_excel(Archivo, sheet, engine='openpyxl')
(Validation).head()
X_val = Validation[['X1', 'X2']]
y_val = Validation[['Y1']]

X_val = X_val.values
y_val = y_val.values

In [6]:
sheet = 'Testing_esc'
Testing = pd.read_excel(Archivo, sheet, engine='openpyxl')
(Testing).head()
X_test = Testing[['X1', 'X2']]
y_test = Testing[['Y1']]

X_test = X_test.values
y_test = y_test.values

# Modelo 1

In [7]:
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense

# Función para crear el Modelo RNN Simple
def crear_modelo():
    modelo = Sequential()
    modelo.add(SimpleRNN(64, input_shape=(ventana, 1)))
    modelo.add(Dense(1, activation='linear'))
    
    return modelo

# Convertir las matrices "X" e "Y" a la forma adecuada para el modelo
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
y_train = y_train.reshape((y_train.shape[0], 1))

# Crear el modelo LSTM
modelo = crear_modelo()

# Compilar y entrenar el modelo
modelo.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
history = modelo.fit(X_train, y_train, epochs=10, batch_size=32,validation_data=(X_val,y_val),verbose=0)

In [8]:
# Hacer predicciones
predictions = modelo.predict(X_test,verbose=0)

In [9]:
# RMSE
sqrt (mean_squared_error (Denormalization(y_test),Denormalization(predictions)))

19.36200729974508

In [10]:
# Generación del dataset
y_test_real = pd.DataFrame(y_test, columns = ["Y_test_real"])
y_test_pred = pd.DataFrame(predictions, columns = ["Y_test_pred"])

y_ventana15_2elems = pd.concat([y_test_real, y_test_pred], axis=1)
y_ventana15_2elems = Denormalization(y_ventana15_2elems)

# Exportar
y_ventana15_2elems.to_excel(nombre1, index_label="Index")

# Modelo 2

In [11]:
# Función para crear el modelo LSTM
def crear_modelo():
    modelo = Sequential()
    modelo.add(LSTM(64, input_shape=(ventana, 1)))  # Capa LSTM con 64 unidades
    modelo.add(Dense(1))  # Capa de salida con 1 unidad
    
    return modelo

# Convertir las matrices "X" e "Y" a la forma adecuada para LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
y_train = y_train.reshape((y_train.shape[0], 1))

# Crear el modelo LSTM
modelo = crear_modelo()

# Compilar y entrenar el modelo
modelo.compile(optimizer='adam', loss='mse', metrics=['accuracy'])
history = modelo.fit(X_train, y_train, epochs=10, batch_size=32,validation_data=(X_val,y_val),verbose=0)
predictions = modelo.predict(X_test,verbose=0)

In [12]:
# RMSE
rmse = sqrt (mean_squared_error (Denormalization(y_test),Denormalization(predictions)))
print(rmse)

19.191857880763354


In [13]:
# Generación del dataset
y_test_real = pd.DataFrame(y_test, columns = ["Y_test_real"])
y_test_pred = pd.DataFrame(predictions, columns = ["Y_test_pred"])

y_ventana15_2elems = pd.concat([y_test_real, y_test_pred], axis=1)
y_ventana15_2elems = Denormalization(y_ventana15_2elems)

# Exportar
y_ventana15_2elems.to_excel(nombre2, index_label="Index")

# Modelo 3

In [14]:
# Función para crear el modelo LSTM
def crear_modelo():
    modelo = Sequential()
    modelo.add(LSTM(128, input_shape=(ventana, 1), return_sequences=True))
    modelo.add(LSTM(128))
    modelo.add(Dense(64, activation='relu'))
    modelo.add(Dense(1, activation='linear'))
    
    return modelo

# Convertir las matrices "X" e "Y" a la forma adecuada para LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
y_train = y_train.reshape((y_train.shape[0], 1))

# Crear el modelo LSTM
modelo = crear_modelo()

# Compilar y entrenar el modelo
modelo.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
history = modelo.fit(X_train, y_train, epochs=10, batch_size=32,validation_data=(X_val,y_val),verbose=0)

# Hacer predicciones
predictions = modelo.predict(X_test,verbose=0)

In [15]:
# RMSE
rmse = sqrt (mean_squared_error (Denormalization(y_test),Denormalization(predictions)))
print(rmse)

11.443219366644962


In [16]:
# Generación del dataset
y_test_real = pd.DataFrame(y_test, columns = ["Y_test_real"])
y_test_pred = pd.DataFrame(predictions, columns = ["Y_test_pred"])

y_ventana15_2elems = pd.concat([y_test_real, y_test_pred], axis=1)
y_ventana15_2elems = Denormalization(y_ventana15_2elems)

In [17]:
# Exportar
y_ventana15_2elems.to_excel(nombre3, index_label="Index")

# Modelo 4

In [18]:
from keras.models import Sequential
from keras.layers import GRU, Dense

# Función para crear el modelo LSTM
def crear_modelo():
    modelo = Sequential()
    modelo.add(GRU(64, input_shape=(ventana, 1)))
    modelo.add(Dense(1, activation='linear'))
    
    return modelo

# Convertir las matrices "X" e "Y" a la forma adecuada para LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
y_train = y_train.reshape((y_train.shape[0], 1))

# Crear el modelo LSTM
modelo = crear_modelo()

# Compilar y entrenar el modelo
modelo.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
history = modelo.fit(X_train, y_train, epochs=10, batch_size=32,validation_data=(X_val,y_val),verbose=0)
# Hacer predicciones
predictions = modelo.predict(X_test,verbose=0)

In [19]:
# RMSE
rmse = sqrt (mean_squared_error (Denormalization(y_test),Denormalization(predictions)))
print(rmse)

19.29464084655125


In [20]:
# Generación del dataset
y_test_real = pd.DataFrame(y_test, columns = ["Y_test_real"])
y_test_pred = pd.DataFrame(predictions, columns = ["Y_test_pred"])

y_ventana15_2elems = pd.concat([y_test_real, y_test_pred], axis=1)
y_ventana15_2elems = Denormalization(y_ventana15_2elems)

# Exportar
y_ventana15_2elems.to_excel(nombre4, index_label="Index")

# Modelo 5

In [21]:
from keras.models import Sequential
from keras.layers import GRU, Dense

def crear_modelo():
    modelo = Sequential()
    modelo.add(GRU(128, input_shape=(ventana, 1), return_sequences=True))
    modelo.add(GRU(128))
    modelo.add(Dense(64, activation='relu'))
    modelo.add(Dense(1, activation='linear'))
    
    return modelo

# Convertir las matrices "X" e "Y" a la forma adecuada para LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
y_train = y_train.reshape((y_train.shape[0], 1))

# Crear el modelo LSTM
modelo = crear_modelo()

# Compilar y entrenar el modelo
modelo.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
history = modelo.fit(X_train, y_train, epochs=10, batch_size=32,validation_data=(X_val,y_val),verbose=0)
# Hacer predicciones
predictions = modelo.predict(X_test,verbose=0)

In [22]:
# RMSE
rmse = sqrt (mean_squared_error (Denormalization(y_test),Denormalization(predictions)))
print(rmse)

13.181152152399372


In [23]:
# Generación del dataset
y_test_real = pd.DataFrame(y_test, columns = ["Y_test_real"])
y_test_pred = pd.DataFrame(predictions, columns = ["Y_test_pred"])

y_ventana15_2elems = pd.concat([y_test_real, y_test_pred], axis=1)
y_ventana15_2elems = Denormalization(y_ventana15_2elems)

# Exportar
y_ventana15_2elems.to_excel(nombre5, index_label="Index")

# Modelo 6

In [24]:
from keras.layers import LSTM, GRU, Dense
num_features = 1

# Función para crear el modelo LSTM y GRU
def crear_modelo():
    modelo = Sequential()
    modelo.add(LSTM(64, input_shape=(ventana, 1), return_sequences=True))
    modelo.add(GRU(64))
    modelo.add(Dense(64, activation='relu'))
    modelo.add(Dense(1, activation='linear'))
    
    return modelo

# Convertir las matrices "X" e "Y" a la forma adecuada para LSTM
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
y_train = y_train.reshape((y_train.shape[0], 1))

# Crear el modelo LSTM
modelo = crear_modelo()

# Compilar y entrenar el modelo
modelo.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])
history = modelo.fit(X_train, y_train, epochs=10, batch_size=32,validation_data=(X_val,y_val),verbose=0)
# Hacer predicciones
predictions = modelo.predict(X_test,verbose=0)

In [25]:
# RMSE
rmse = sqrt (mean_squared_error (Denormalization(y_test),Denormalization(predictions)))
print(rmse)

10.636067070309219


In [26]:
# Generación del dataset
y_test_real = pd.DataFrame(y_test, columns = ["Y_test_real"])
y_test_pred = pd.DataFrame(predictions, columns = ["Y_test_pred"])

y_ventana15_2elems = pd.concat([y_test_real, y_test_pred], axis=1)
y_ventana15_2elems = Denormalization(y_ventana15_2elems)

# Exportar
y_ventana15_2elems.to_excel(nombre6, index_label="Index")